In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-05-15 10:58:58--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-15 10:58:58--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-15 10:58:58--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [69]:
!pip install emoji

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=09410d55a628fa85fe834f4a64a363cce25df8ea17a0950c20a7d1349466100e
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import emoji

In [0]:
import numpy as np

In [0]:
from tensorflow.keras.layers import Embedding

In [0]:
from tensorflow.keras.layers import LSTM,Input,Dense,Dropout

In [0]:
from tensorflow.keras.models import Model

In [0]:
import pandas as pd

In [0]:
from sklearn.preprocessing import LabelBinarizer

In [0]:
!unzip -q glove.6B.zip

In [0]:
w2v = dict()
w2i = dict()
i2w = dict()
words = set()

In [0]:
with open('glove.6B.50d.txt') as f:
  for i in f:
    x = i.strip().split()
    words.add(x[0])
    w2v[x[0]] = np.array(x[1:],dtype = np.float64)
  for i,j in enumerate(sorted(words)):
    w2i[j] = i
    i2w[i] = j

In [0]:
def sent2idx(x,w2i,maxm):
  n = x.shape[0]
  ids = np.zeros((n,maxm))
  for i in range(n):
    words = x[i].lower().split()
    for j,k in enumerate(words):
      ids[i,j] = w2i[k]

  return ids

In [0]:
def embed_layer(w2v,w2i):
  lens = len(w2i)+1
  dims = 50

  emb_mx = np.zeros((lens,dims))

  for w,i in w2i.items():
    emb_mx[i,] = w2v[w]
  
  emb_layer = Embedding(lens,dims,trainable = False)
  emb_layer.build((None,))
  emb_layer.set_weights([emb_mx])

  return emb_layer

In [0]:
def model(in_shape,w2v,w2i):
  inputs = Input(in_shape,dtype = 'int32')
  emb_layer = embed_layer(w2v,w2i)
  emb = emb_layer(inputs)
  x = LSTM(128,return_sequences=True)(emb)
  x = Dropout(0.4)(x)
  x = LSTM(128)(x)
  x = Dropout(0.4)(x)
  x = Dense(5,activation = 'softmax')(x)
  model = Model(inputs,x)
  return model

In [0]:
train = pd.read_csv('train_emoji.csv',header = None)
test = pd.read_csv('tesss.csv',header = None)

In [0]:
lb = LabelBinarizer()

In [0]:
trainx = np.array(train[0])
trainy = lb.fit_transform(train[1])

In [0]:
maxm = len(max(trainx,key = len).split())

In [0]:
model = model((maxm,),w2v,w2i)

In [0]:
model.compile('adam','categorical_crossentropy',['accuracy'])

In [0]:
tx = sent2idx(trainx,w2i,maxm)

In [65]:
model.fit(tx,trainy,batch_size=32,epochs = 100)

Epoch 1/100
5/5 [==============================] - 0s 18ms/step - loss: 1.5409 - accuracy: 0.3182
Epoch 2/100
5/5 [==============================] - 0s 17ms/step - loss: 1.4931 - accuracy: 0.3939
Epoch 3/100
5/5 [==============================] - 0s 18ms/step - loss: 1.4637 - accuracy: 0.3636
Epoch 4/100
5/5 [==============================] - 0s 18ms/step - loss: 1.3707 - accuracy: 0.4167
Epoch 5/100
5/5 [==============================] - 0s 18ms/step - loss: 1.3307 - accuracy: 0.4470
Epoch 6/100
5/5 [==============================] - 0s 19ms/step - loss: 1.1950 - accuracy: 0.5530
Epoch 7/100
5/5 [==============================] - 0s 18ms/step - loss: 1.0268 - accuracy: 0.6364
Epoch 8/100
5/5 [==============================] - 0s 18ms/step - loss: 0.8668 - accuracy: 0.6591
Epoch 9/100
5/5 [==============================] - 0s 19ms/step - loss: 0.8989 - accuracy: 0.6515
Epoch 10/100
5/5 [==============================] - 0s 19ms/step - loss: 0.7379 - accuracy: 0.7576
Epoch 11/100
5/5 [=

In [0]:
testx = np.array(test[0])
testy = lb.fit_transform(test[1])

In [67]:
_,acc = model.evaluate(sent2idx(testx,w2i,maxm),testy)

2/2 [==============================] - 0s 6ms/step - loss: 0.8414 - accuracy: 0.8571


In [68]:
print(acc)

0.8571428656578064


In [0]:
emoji_dict = {0:':heart:',1:':baseball:',2:':smile:',3:':disappointed:',4:':fork_and_knife:'}

In [0]:
preds = model.predict(sent2idx(testx,w2i,maxm))

In [0]:
preds = preds.argmax(axis = 1)

In [0]:
data = pd.DataFrame()
data['Text'] = test[0]
data['Original'] = [emoji.emojize(emoji_dict[x],use_aliases=True) for x in test[1]]
data['Predicted'] = [emoji.emojize(emoji_dict[x],use_aliases=True) for x in preds.ravel()]

In [83]:
data

,Text,Original,Predicted
0,I want to eat\t,🍴,🍴
1,he did not answer\t,😞,😞
2,he got a very nice raise\t,😄,❤
3,she got me a nice present\t,😄,😄
4,ha ha ha it was so funny\t,😄,😄
5,he is a good friend\t,😄,😄
6,I am upset\t,😞,😞
7,We had such a lovely dinner tonight\t,😄,😄
8,where is the food\t,🍴,🍴
9,Stop making this joke ha ha ha\t,😄,😄
